In [31]:
import numpy as np
import pandas as pd

df = pd.read_csv("daily_rainfall_jalgaon_chalisgaon_talegaon_2022.csv")

def compute_runoff(rain):
    if(rain < 0):
        return None
    if(rain < 25):
        return 0.2*rain
    if(rain < 50):
        return 0.3*rain
    if(rain < 75):
        return 0.4*rain
    if(rain < 100):
        return 0.5*rain
    return 0.7*rain

def compute_data(prev_sm, rain, soil_quality, ):
    gamma = 0.2 if(soil_quality == "deep") else 0.4
    C = 100 if(soil_quality == "deep") else 42
    runoff = compute_runoff(rain)

    crop_uptake = min(prev_sm + rain - runoff, 4)
    infiltration = prev_sm + rain - runoff - crop_uptake
    
    excess = max(infiltration - C, 0)
    sm_plus_gw = infiltration - excess

    sm = sm_plus_gw/(1+gamma)
    gw = sm_plus_gw - sm

    return rain, runoff + excess, crop_uptake, sm, gw

In [32]:
deep_file = open("output/deep_sm.csv", "w")
shallow_file = open("output/shallow_sm.csv", "w")
deep_file.write("date,rain_mm,total_runoff_mm,crop_water_uptake_mm,soil_moisture_mm,percolation_to_groundwater_mm\n")
shallow_file.write("date,rain_mm,total_runoff_mm,crop_water_uptake_mm,soil_moisture_mm,percolation_to_groundwater_mm\n")

# assuming initial soil moisture is 0
deep_prev_sm = 0
shallow_prev_sm = 0

for i in range(len(df["rain_mm"])):
    deep_data = compute_data(deep_prev_sm, df["rain_mm"][i], "deep")
    shallow_data = compute_data(shallow_prev_sm, df["rain_mm"][i], "shallow")
    
    deep_file.write(df["date"][i])
    for j in range(len(deep_data)):
        deep_file.write("," + str(deep_data[j]))
    
    shallow_file.write(df["date"][i])
    for j in range(len(shallow_data)):
        shallow_file.write("," + str(shallow_data[j]))
    
    deep_prev_sm = 

<class 'str'>
